#Gold Layer Scripting : Business-Level Data Modeling

This script creates views for the Gold layer in the data lakehouse. The Gold layer represents the final dimension and fact tables.Each view performs transformations and combines data from the Silver layer to produce a clean, enriched, and business-ready dataset, that answer real business questions.

### The Transformation Logic :
The Sales Fact Table captures transactional sales data and links it with customer and product dimensions. It represents measurable business events and serves as the foundation for revenue analysis, performance tracking, and KPI reporting.

In [0]:
querry = """
select
    sd.order_number,
    pr.product_key,
    cu.customer_key,
    sd.order_date,
    sd.ship_date,
    sd.due_date,
    sd.sales_amount,
    sd.quantity,
    sd.price

from 
    bike_lakehouse.silver.crm_sales sd
    
left join 
    bike_lakehouse.gold.dim_products pr
on 
    sd.product_number = pr.product_number

left join
    bike_lakehouse.gold.dim_customers cu
on 
    sd.customer_id = cu.customer_id;

"""


In [0]:
df = spark.sql(querry)

In [0]:
df.limit(10).display()

order_number,product_key,customer_key,order_date,ship_date,due_date,sales_amount,quantity,price
SO43697,null,10769,2010-12-29,2011-01-05,2011-01-10,3578,1,3578.0
SO43698,null,17390,2010-12-29,2011-01-05,2011-01-10,3400,1,3400.0
SO43699,null,14864,2010-12-29,2011-01-05,2011-01-10,3400,1,3400.0
SO43700,null,3502,2010-12-29,2011-01-05,2011-01-10,699,1,699.0
SO43701,null,4,2010-12-29,2011-01-05,2011-01-10,3400,1,3400.0
SO43702,null,16646,2010-12-30,2011-01-06,2011-01-11,3578,1,3578.0
SO43703,null,5625,2010-12-30,2011-01-06,2011-01-11,3578,1,3578.0
SO43704,null,6,2010-12-30,2011-01-06,2011-01-11,3375,1,3375.0
SO43705,null,12,2010-12-30,2011-01-06,2011-01-11,3400,1,3400.0
SO43706,null,16622,2010-12-31,2011-01-07,2011-01-12,3578,1,3578.0


### Writing Gold Table
Persist the cleaned DataFrame as a Delta table in the Gold layer.


In [0]:
df.write.mode('overwrite').format('delta').saveAsTable('bike_lakehouse.gold.fact_sales')

### Sanity checks of Gold table
Quickly check the result of transformations, before moving forward with the dataFrame

In [0]:
%sql
select count(*) from bike_lakehouse.gold.fact_sales where product_key is null  ;

count(*)
3241
